In [35]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt 
#import seaborn as sns
#import plotly.express as px
from pprint import pprint

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import lightgbm as lgb

from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

from functions import df_engineered, r2rmse_scores

import pickle
import time

In [2]:
feature_selection = ['ExterQual', 'BsmtQual', 'KitchenQual', 'OverallQual', 
                    'GrLivArea', 'TotalBsmtSF', 'GarageArea', 'FullBath', 
                    'HouseAge', 'TotRmsAbvGrd', 'OverallCond',  'FlrSF1st', 'FlrSF2nd',
                    'Fireplaces', 'HasFireplace','LotFrontage', 'LotArea', 'MSSubClass', 'GoodGarageType', 
                    'BsmtUnfSF', 'Zone', 'Location', 'CulDSac', 'ExQual', 'RemodAge', 'LargerHouse', 
                    'Remod', 'ExBsmtQual', 'TwoStory', 'TotalSF', 'RoadRail', 'ExKitchen', 'CentralAir',
                    'TotalBath', 'Exterior1st_top']
                        

In [3]:
df = pd.read_csv('data/Ames_Housing_Price_Data.csv', index_col=0).iloc[:,1:]
print('df shape:', df.shape)

data = df_engineered(df, Reg=False)

df shape: (2580, 80)


In [4]:
X_train, X_test, y_train, y_test = train_test_split(data[feature_selection], data['SalePrice'], test_size=0.30, random_state=42)

## Random Forest

#### Untuned model

In [5]:
rf_untuned = GradientBoostingRegressor(random_state=42)
rf_untuned.fit(X_train, y_train)

r2rmse_scores(rf_untuned, X_train, y_train)

--------------------------------------------------
5-fold Cross Validation Scoring
Mean R^2 score: 0.9308136758403526
Mean RMSE score: 18403.94321454822
--------------------------------------------------


In [6]:
yhat_train = rf_untuned.predict(X_train)
yhat_test = rf_untuned.predict(X_test)

train_mse_non_log = mean_squared_error(y_train, yhat_train, squared=False)
test_mse_non_log = mean_squared_error(y_test, yhat_test, squared=False)

print(train_mse_non_log)
print(test_mse_non_log)

12666.367773469008
20022.1534682249


#### Tuned Model

In [7]:
n_estimators = [int(x) for x in np.linspace(start = 90, stop = 100, num = 10)]
max_features = ['sqrt', 'log2']
max_depth = [int(x) for x in np.linspace(3, 10, num = 7)]
min_samples_split = [2, 5, 10, 15]
min_samples_leaf = [4, 8, 12, 15]
bootstrap = [True, False]


random_grid = [{'n_estimators': n_estimators,
               'max_features': max_features,
               'bootstrap': bootstrap,
               'max_features':max_features,
               'min_samples_split': min_samples_split},
               {
               'n_estimators': n_estimators,
               'max_features': max_features,
               'bootstrap': bootstrap,   
               'max_features':max_features,
               'min_samples_leaf': min_samples_leaf}]

pprint(random_grid)

[{'bootstrap': [True, False],
  'max_features': ['sqrt', 'log2'],
  'min_samples_split': [2, 5, 10, 15],
  'n_estimators': [90, 91, 92, 93, 94, 95, 96, 97, 98, 100]},
 {'bootstrap': [True, False],
  'max_features': ['sqrt', 'log2'],
  'min_samples_leaf': [4, 8, 12, 15],
  'n_estimators': [90, 91, 92, 93, 94, 95, 96, 97, 98, 100]}]


In [8]:
rf = RandomForestRegressor(random_state=42)

rf_random = RandomizedSearchCV(estimator=rf, 
                               param_distributions=random_grid, 
                               n_iter=100, 
                               cv=3, 
                               verbose=1, 
                               random_state=42, 
                               n_jobs = -1)

rf_random.fit(X_train, y_train)

r2rmse_scores(rf_random, X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Fitting 3 folds for each of 100 candidates, totalling 300 fits
--------------------------------------------------
5-fold Cross Validation Scoring
Mean R^2 score: 0.9193211143372606
Mean RMSE score: 19911.49854168351
--------------------------------------------------


In [9]:
yhat_train = rf_random.predict(X_train)
yhat_test = rf_random.predict(X_test)

train_mse_non_log = mean_squared_error(y_train, yhat_train, squared=False)
test_mse_non_log = mean_squared_error(y_test, yhat_test, squared=False)

print(train_mse_non_log)
print(test_mse_non_log)

198.1425898595034
19991.636674466707


#### Save best model

In [10]:
#file_name = "rf_reg_{}.pkl".format(time.strftime("%Y%m%d-%H.%M.%S"))

# save
#pickle.dump(rf_random, open(file_name, "wb"))

# load
file_name='rf_reg_20230719-14.40.17.pkl'
rf_model_loaded = pickle.load(open(file_name, "rb"))

In [11]:
yhat_train = rf_model_loaded.predict(X_train)
yhat_test = rf_model_loaded.predict(X_test)

train_mse_non_log = mean_squared_error(y_train, yhat_train, squared=False)
test_mse_non_log = mean_squared_error(y_test, yhat_test, squared=False)

print(train_mse_non_log)
print(test_mse_non_log)

198.0014632889261
19859.061651746244


## XGBoost sklearn

#### Untuned model

In [12]:
xgb_untuned = GradientBoostingRegressor(random_state=42)
xgb_untuned.fit(X_train, y_train)

r2rmse_scores(xgb_untuned, X_train, y_train)

--------------------------------------------------
5-fold Cross Validation Scoring
Mean R^2 score: 0.9308136758403526
Mean RMSE score: 18403.94321454822
--------------------------------------------------


In [13]:
yhat_train = xgb_untuned.predict(X_train)
yhat_test = xgb_untuned.predict(X_test)

train_mse_non_log = mean_squared_error(y_train, yhat_train, squared=False)
test_mse_non_log = mean_squared_error(y_test, yhat_test, squared=False)

print(train_mse_non_log)
print(test_mse_non_log)

12666.367773469008
20022.1534682249


#### Tuned Model

In [14]:
n_estimators = [int(x) for x in np.linspace(start = 160, stop = 180, num = 10)]
max_features = ['sqrt', 'log2']
max_depth = [int(x) for x in np.linspace(3, 5, num = 3)]
min_samples_split = [8, 9, 10, 11, 12]
min_samples_leaf = [3,4,5]
learning_rate = [0.08, 0.09, 0.1, 0.12, 0.13]
alpha = [0.15, 0.2, 0.25]
subsample = [0.6, 0.8, 0.9]

random_grid = [{'n_estimators': n_estimators,
               'max_features': max_features,
               'learning_rate':learning_rate,
               'alpha':alpha,
               'max_depth':max_depth,
               'subsample':subsample,
               'min_samples_split': min_samples_split},
               {
               'n_estimators': n_estimators,
               'max_features': max_features, 
               'learning_rate':learning_rate,
               'alpha':alpha,
               'max_depth':max_depth,
               'subsample':subsample,
               'min_samples_leaf': min_samples_leaf}]

pprint(random_grid)

[{'alpha': [0.15, 0.2, 0.25],
  'learning_rate': [0.08, 0.09, 0.1, 0.12, 0.13],
  'max_depth': [3, 4, 5],
  'max_features': ['sqrt', 'log2'],
  'min_samples_split': [8, 9, 10, 11, 12],
  'n_estimators': [160, 162, 164, 166, 168, 171, 173, 175, 177, 180],
  'subsample': [0.6, 0.8, 0.9]},
 {'alpha': [0.15, 0.2, 0.25],
  'learning_rate': [0.08, 0.09, 0.1, 0.12, 0.13],
  'max_depth': [3, 4, 5],
  'max_features': ['sqrt', 'log2'],
  'min_samples_leaf': [3, 4, 5],
  'n_estimators': [160, 162, 164, 166, 168, 171, 173, 175, 177, 180],
  'subsample': [0.6, 0.8, 0.9]}]


In [15]:
xgb = GradientBoostingRegressor(random_state=42)

xgb_random = RandomizedSearchCV(estimator=xgb, 
                               param_distributions=random_grid, 
                               n_iter=400, 
                               cv=3, 
                               verbose=1, 
                               random_state=42, 
                               n_jobs = -1)

xgb_random.fit(X_train, y_train)

r2rmse_scores(xgb_random, X_train, y_train)

Fitting 3 folds for each of 400 candidates, totalling 1200 fits
Fitting 3 folds for each of 400 candidates, totalling 1200 fits
Fitting 3 folds for each of 400 candidates, totalling 1200 fits
Fitting 3 folds for each of 400 candidates, totalling 1200 fits
Fitting 3 folds for each of 400 candidates, totalling 1200 fits
Fitting 3 folds for each of 400 candidates, totalling 1200 fits
Fitting 3 folds for each of 400 candidates, totalling 1200 fits
Fitting 3 folds for each of 400 candidates, totalling 1200 fits
Fitting 3 folds for each of 400 candidates, totalling 1200 fits
Fitting 3 folds for each of 400 candidates, totalling 1200 fits
Fitting 3 folds for each of 400 candidates, totalling 1200 fits
--------------------------------------------------
5-fold Cross Validation Scoring
Mean R^2 score: 0.9337562309965988
Mean RMSE score: 17992.595343969217
--------------------------------------------------


In [16]:
yhat_train = xgb_random.predict(X_train)
yhat_test = xgb_random.predict(X_test)

train_mse_non_log = mean_squared_error(y_train, yhat_train, squared=False)
test_mse_non_log = mean_squared_error(y_test, yhat_test, squared=False)

print(train_mse_non_log)
print(test_mse_non_log)

12379.414691259073
18063.257706544446


#### Save best model

In [33]:
#file_name = "xgb_reg_{}.pkl".format(time.strftime("%Y%m%d-%H.%M.%S"))

# save
#pickle.dump(xgb_random, open(file_name, "wb"))

# load
file_name='xgb_reg_20230719-13.12.55.pkl'
xgb_model_loaded = pickle.load(open(file_name, "rb"))

In [38]:
yhat_train = xgb_model_loaded.predict(X_train)
yhat_test = xgb_model_loaded.predict(X_test)

train_mse_non_log = mean_squared_error(y_train, yhat_train, squared=False)
test_mse_non_log = mean_squared_error(y_test, yhat_test, squared=False)

print(train_mse_non_log)
print(test_mse_non_log)

9857.474442438353
16906.98734415023


## LightGBM

#### Untuned model

In [19]:
lgbm = lgb.LGBMRegressor()
lgbm.fit(X_train, y_train)

LGBMRegressor()

In [20]:
yhat_train = lgbm.predict(X_train)
yhat_test = lgbm.predict(X_test)

train_mse_non_log = mean_squared_error(y_train, yhat_train, squared=False)
test_mse_non_log = mean_squared_error(y_test, yhat_test, squared=False)

print(train_mse_non_log)
print(test_mse_non_log)

9024.91978810812
21787.229150430783


#### Tuned Model

In [21]:
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 190, num = 10)]
max_depth = [int(x) for x in np.linspace(2, 5, num = 3)]
learning_rate = [0.08, 0.1, 0.2, 0.3]
alpha = [0.1, 0.2, 0.3, 0.4]
subsample = [0.6, 0.8, 0.9]
min_child_samples = [3,4, 5, 6,7]
colsample_bytree = [0.7, 0.8, 0.9]

random_grid = [{'n_estimators': n_estimators,
               'learning_rate':learning_rate,
               'alpha':alpha,
               'max_depth':max_depth,
               'subsample':subsample,
               'min_child_samples':min_child_samples,
               'colsample_bytree':colsample_bytree}]

random_grid

[{'n_estimators': [10, 30, 50, 70, 90, 110, 130, 150, 170, 190],
  'learning_rate': [0.08, 0.1, 0.2, 0.3],
  'alpha': [0.1, 0.2, 0.3, 0.4],
  'max_depth': [2, 3, 5],
  'subsample': [0.6, 0.8, 0.9],
  'min_child_samples': [3, 4, 5, 6, 7],
  'colsample_bytree': [0.7, 0.8, 0.9]}]

In [22]:
lgbm = lgb.LGBMRegressor(objective='regression', 
                                random_state=42)

lgbm_random = RandomizedSearchCV(estimator=lgbm, 
                      param_distributions=random_grid,
                      n_iter=100, 
                      cv=3, 
                      verbose=1, 
                      random_state=42, 
                      n_jobs = -1,
                      scoring='neg_root_mean_squared_error')

lgbm_random.fit(X_train, y_train)

r2rmse_scores(lgbm_random, X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Fitting 3 folds for each of 100 candidates, totalling 300 fits
--------------------------------------------------
5-fold Cross Validation Scoring
Mean R^2 score: 0.934047280268391
Mean RMSE score: 17959.81347978003
--------------------------------------------------


In [23]:
yhat_train = lgbm_random.predict(X_train)
yhat_test = lgbm_random.predict(X_test)

train_mse_non_log = mean_squared_error(y_train, yhat_train, squared=False)
test_mse_non_log = mean_squared_error(y_test, yhat_test, squared=False)

print(train_mse_non_log)
print(test_mse_non_log)

11960.92188118916
18519.067420985775


In [24]:
lgbm_random.best_params_

{'subsample': 0.9,
 'n_estimators': 130,
 'min_child_samples': 4,
 'max_depth': 3,
 'learning_rate': 0.1,
 'colsample_bytree': 0.7,
 'alpha': 0.4}

#### Save best model

In [30]:
#file_name = "lgbm_reg_{}.pkl".format(time.strftime("%Y%m%d-%H.%M.%S"))

# save
#pickle.dump(lgbm_random, open(file_name, "wb"))

# load
file_name='lgbm_reg_20230719-15.03.27.pkl'
lgbm_model_loaded = pickle.load(open(file_name, "rb"))

In [31]:
yhat_train = lgbm_model_loaded.predict(X_train)
yhat_test = lgbm_model_loaded.predict(X_test)

train_mse_non_log = mean_squared_error(y_train, yhat_train, squared=False)
test_mse_non_log = mean_squared_error(y_test, yhat_test, squared=False)

print(train_mse_non_log)
print(test_mse_non_log)

11960.92188118916
18519.067420985775
